# Model Management
## Azure ML and MLFlow tracking

In [1]:
# Deploy model
# Connect to workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id="49b5441f-dda4-47a9-81c4-13272430f4ff",
    resource_group_name="rg-pooya120-dev",
    workspace_name="amlwpooya120dev",
)

In [2]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
import os

score_dir = './score'
os.makedirs(score_dir, exist_ok=True)

In [3]:
%%writefile {score_dir}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - azureml-defaults>=1.42.0
    - azureml-inference-server-http
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0
    - psutil>=5.8,<5.9
    - tqdm>=4.59,<4.60
    - ipykernel~=6.0
    - matplotlib

Overwriting ./score/conda.yml


In [4]:
from azure.ai.ml.entities import Environment

custom_env_name = "bike-share-env"

env = Environment(
    name=custom_env_name,
    description="Custom environment for Bike Share pipeline",
    tags={"scikit-learn": "0.24.2"},
    conda_file= "score/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)
env = ml_client.environments.create_or_update(env)

print(
    f"Environment {env.name} registered to workspace. "
    f"Environment version is {env.version}."
)

Environment bike-share-env registered to workspace. Environment version is 3.


## Endpoints and Deployments

- An Endpoint is an HTTP server that allows interaction with the model(s).
- An endpoint may serve multiple models.
- Each model must be deployed in order for the endpoint to serve it.
- Hence, one Endpoint can have many Deployments.
- Blue-Green deployment and traffic management can be handled here or using az CLI.

In [ ]:
# Create an endpoint name
endpoint_name = "my-endpoint2"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="Bike share endpoint",
    auth_mode="key",
)

ml_client.online_endpoints.begin_create_or_update(endpoint, ).result()

In [ ]:
# grab the model to be deployed and the serving environment
model = ml_client.models.get('sklearn.ensemble._forest.RandomForestRegressor',1)
env = ml_client.environments.get('aml-scikit-learn', 2)

In [ ]:

deployment = ManagedOnlineDeployment(
    name="blue2",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="./score", scoring_script="score.py"
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
    app_insights_enabled=True,
)
ml_client.online_deployments.begin_create_or_update(deployment=deployment, ).result()